In [1]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from ensemble_boxes import weighted_boxes_fusion

/home/hungld11/anaconda3/envs/python_3.7/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pickle
def load_dict(name):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [3]:
submission_study_output = []
submission_image_output = []
study_pred = torch.load("/home/hungld11/Documents/SIIM COVID DETECTION/results/study_level/efficientnet-b5_512_UnetDecoder_aux_fold0_1_2_3_4_test_pred_8tta_1.pth")['pred_dict']
# image_preds = torch.load("/home/hungld11/Documents/SIIM COVID DETECTION/results/image_level/resnet101d_512_fold_3_4_0_1_2_test_pred_1.pth")
test_df = pd.read_csv("/home/hungld11/Documents/SIIM COVID DETECTION/dataset/test_meta.csv")
classes = ["negative", "typical", "indeterminate", "atypical"]

In [5]:
submission_study_output = []
for studyid, grp in tqdm(test_df.groupby('studyid')):
    preds = []
    for _, row in grp.iterrows():
        pred =  study_pred[row['imageid']]     
        preds.append(pred)
        # boxes, scores, labels, img_width, img_height = image_preds[row['imageid']]
        
        boxes, scores, labels, img_width, img_height = load_dict("/home/hungld11/Documents/SIIM COVID DETECTION/results/image_level_pkl/{}.pkl".format(row['imageid']))
        boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=0.6)
        assert np.mean(labels) == 0
        boxes = boxes.clip(0,1)
        
        boxes[:,[0,2]] = boxes[:,[0,2]]*float(img_width)
        boxes[:,[1,3]] = boxes[:,[1,3]]*float(img_height)
        
        neg_image_pred = 'none {} 0 0 1 1'.format(pred[0])
        opacity_image_pred = []
        for box, score in zip(boxes, scores):
            opacity_image_pred.append('opacity {} {} {} {} {}'.format(score, box[0], box[1], box[2],box[3]))
        image_pred = ' '.join([neg_image_pred] + opacity_image_pred)
        submission_image_output.append(['{}_image'.format(row['imageid']), image_pred])
        
    preds = np.array(preds)
    preds = np.mean(preds, axis=0)
        
    study_preds = []
    for clsidx, clsname in enumerate(classes):
        study_preds.append('{} {} 0 0 1 1'.format(classes[clsidx], preds[clsidx]))
    study_preds = ' '.join(study_preds)
    submission_study_output.append(['{}_study'.format(studyid), study_preds])



100%|██████████| 1214/1214 [00:11<00:00, 104.04it/s]


In [6]:
submission_output = submission_study_output + submission_image_output
sub_df = pd.DataFrame(data = np.array(submission_output), columns=['id','PredictionString'])

In [7]:
sub_df

,id,PredictionString
0,00188a671292_study,negative 0.8642578125 0 0 1 1 typical 0.033630...
1,004bd59708be_study,negative 0.00287628173828125 0 0 1 1 typical 0...
2,00508faccd39_study,negative 0.58251953125 0 0 1 1 typical 0.17382...
3,006486aa80b2_study,negative 0.2203369140625 0 0 1 1 typical 0.347...
4,00655178fdfc_study,negative 0.429443359375 0 0 1 1 typical 0.2480...
...,...,...
2472,064b37b01cd2_image,none 0.1510009765625 0 0 1 1 opacity 0.3777035...
2473,02eceb0fc405_image,none 0.1619873046875 0 0 1 1 opacity 0.6494870...
2474,16a0d37747d7_image,none 0.00049591064453125 0 0 1 1 opacity 0.959...
2475,aa55fc6c616c_image,none 0.61669921875 0 0 1 1 opacity 0.505420223...


In [ ]:
sub_df.to_csv("/home/hungld11/Documents/SIIM COVID DETECTION/results/public_submission_2.csv", index=False)